## Example: Converting a Segmentation Stack + geff for Use with TrackGardener

This notebook provides a practical example of converting a segmentation stack and [geff](https://github.com/live-image-tracking-tools/geff) to a TrackGardener database using the configuration file `Fluo-N2DL-HeLa-01_config_geff.yaml`.

In this workflow, the segmentation stack provides the exact segmentation masks, while the geff file supplies only the graph structure. We assume that the geff file specifies both `t` and `segm_id` for each entry. We verify that, in the segmentation array (labeled stack), there is an object at frame t labeled with the value segm_id. For details on additional tests, please refer to the documentation.

All signals are calculated as specified in the config file. Please note that this process may take a significant amount of time for large datasets.

You can reuse the configuration file demonstrated here with the TrackGardener plugin itself. Just be sure to specify the correct paths to your imaging dataset and the TrackGardener database within the config file. Using absolute (rather than relative as for the purpose of this example) paths is the safest choice to ensure your files are correctly located.

In [1]:
import dask.array as da
import yaml

from track_gardener.converters.geff_2_gardener import segm_geff_to_db
from track_gardener.converters.validator_seg_id import validate_geff_seg_ids

In [ ]:
geff_group_path = "Fluo-N2DL-HeLa-01_sample.geff"
segmentation_path = "Fluo-N2DL-HeLa-01_segm_random.zarr/labels"
config_path = 'Fluo-N2DL-HeLa-01_config_geff.yaml'

In [3]:
p, details = validate_geff_seg_ids(
    geff_group_path = geff_group_path,
    segmentation_path = segmentation_path,
    seg_id_field = "segm_id",
    check_xy_position = True,
    check_extra_segmentations = True)

print(f'Test passed: {p}, details: {details}.')

2025-07-25 11:12:21.269 | INFO     | track_gardener.converters.validator_seg_id:validate_geff_seg_ids:31 - Opening GEFF group from: Fluo-N2DL-HeLa-01_sample.geff
2025-07-25 11:12:21.334 | INFO     | track_gardener.converters.validator_seg_id:validate_geff_seg_ids:52 - Beginning node validation...
2025-07-25 11:12:22.468 | INFO     | track_gardener.converters.validator_seg_id:validate_geff_seg_ids:93 - Checking for extra segmentations not referenced in GEFF...
2025-07-25 11:12:22.746 | SUCCESS  | track_gardener.converters.validator_seg_id:validate_geff_seg_ids:112 - All validations passed successfully.


Test passed: True, details: {'node_issues': [], 'extra_segments': {}}.


In [4]:
# read in configuration file
with open(config_path) as file:
    config = yaml.safe_load(file)

# get segmentation array
segm_array = da.from_zarr(segmentation_path)

In [5]:
segm_geff_to_db(segm_array, config, geff_group_path)

  0%|          | 0/45 [00:00<?, ?it/s]2025-07-25 11:12:29.753 | INFO     | track_gardener.converters.track_array_2_gardener:convert_labeled_frame_to_cells:89 - Found 2 labeled objects at t=0
2025-07-25 11:12:29.800 | INFO     | track_gardener.converters.track_array_2_gardener:convert_labeled_frame_to_cells:89 - Found 2 labeled objects at t=1
  4%|▍         | 2/45 [00:00<00:02, 19.23it/s]2025-07-25 11:12:29.871 | INFO     | track_gardener.converters.track_array_2_gardener:convert_labeled_frame_to_cells:89 - Found 2 labeled objects at t=2
2025-07-25 11:12:29.944 | INFO     | track_gardener.converters.track_array_2_gardener:convert_labeled_frame_to_cells:89 - Found 3 labeled objects at t=3
  9%|▉         | 4/45 [00:00<00:02, 14.95it/s]2025-07-25 11:12:30.023 | INFO     | track_gardener.converters.track_array_2_gardener:convert_labeled_frame_to_cells:89 - Found 3 labeled objects at t=4
2025-07-25 11:12:30.108 | INFO     | track_gardener.converters.track_array_2_gardener:convert_labeled_fra